# ALU

## 1. 编码
### 1.1 浮点数与定点数之间转换

In [ ]:
#浮点数转定点数
import math

def float_to_fixed(flt: float, maxbits: int, factor: int, clip: bool = True) -> int:
  """
  将浮点数转换为定点数。

  Args:
    flt: 要转换的浮点数。
    maxbits: 定点数的最大位数。
    factor: 定点数的因子。
    clip: 是否裁剪定点数的值。如果 `clip` 为 `true`，则定点数的值将被限制在 `min` 和 `max` 之间。

  Returns:
    定点数的值。
  """

  max = math.pow(2, maxbits - 1) - 1
  min = -max
  fixed = flt * factor

  if clip:
    if fixed > max:
      fixed = max
    elif fixed < min:
      fixed = min

  return int(fixed)

def fixed_to_float(fixed: int, factor: int) -> float:
  """
  将定点数转换为浮点数。

  Args:
    fixed: 要转换的定点数。
    factor: 定点数的因子。

  Returns:
    浮点数的值。
  """

  return fixed / factor

def float_to_fixed_vector(flt: list[float], maxbits: int, factor: int, clip: bool = True) -> list[int]:
  """
  将浮点数数组转换为定点数数组。

  Args:
    flt: 要转换的浮点数数组。
    maxbits: 定点数的最大位数。
    factor: 定点数的因子。
    clip: 是否裁剪定点数的值。如果 `clip` 为 `true`，则定点数的值将被限制在 `min` 和 `max` 之间。

  Returns:
    定点数数组。
  """

  fixed_vector = []
  for f in flt:
    fixed_vector.append(float_to_fixed(f, maxbits, factor, clip))

  return fixed_vector

def fixed_to_float_vector(fixed: list[int], factor: int) -> list[float]:
  """
  将定点数数组转换为浮点数数组。

  Args:
    fixed: 要转换的定点数数组。
    factor: 定点数的因子。

  Returns:
    浮点数数组。
  """

  flt_vector = []
  for f in fixed:
    flt_vector.append(fixed_to_float(f, factor))

  return flt_vector

In [149]:
#test
flt = 123.456
maxbits = 8
factor = 1

fixed = float_to_fixed(flt, maxbits, factor)
print(fixed)
# 将定点数转换为浮点数
flt = fixed_to_float(fixed, factor)
print(flt)
# 将浮点数数组转换为定点数数组
flt_vector = [123.456, 789.012]

fixed_vector = float_to_fixed_vector(flt_vector, maxbits, factor)
print(fixed_vector)
# 将定点数数组转换为浮点数数组
flt_vector = fixed_to_float_vector(fixed_vector, factor)
print(flt_vector)

123
123.0
[123, 127]
[123.0, 127.0]


### 1.2 将整数转换为二进制数组

In [153]:
def extract_bits_with_shift(num: int) -> list:
  """
  使用移位提取数字的二进制位，并存储到一个数组中。

  Args:
    num: 要提取二进制位的数字。

  Returns:
    二进制位数组。
  """

  bits = []
  for i in range(8):
    bits.append(num >> i & 1)

  return bits


if __name__ == "__main__":
  num1 = -10
  bits1 = extract_bits_with_shift(num1)
  print(bits1)
  num2 = 2
  bits2 = extract_bits_with_shift(num2)
  print(bits2)

[0, 1, 1, 0, 1, 1, 1, 1]
[0, 1, 0, 0, 0, 0, 0, 0]


### 1.3 将二进制数组转换为整数

In [154]:
def binary_to_decimal(binary: list) -> int:
  """
  将二进制数组转换为十进制。

  Args:
    binary: 二进制数组。

  Returns:
    十进制数。
  """

  length = len(binary)
  decimal = 0
  for i in range(length):
    decimal |= binary[i] << i
  return decimal


if __name__ == "__main__":
  binary = [0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1]
  decimal = binary_to_decimal(binary)
  print(decimal)

44400


### 1.4 二进制补码转十进制

In [155]:
def to_complement(num):
    nb_samples = len(num)
    sum_result = 0

    for i in range(nb_samples - 1):
        sum_result += num[i] << i

    if num[nb_samples - 1]:
        sum_result -= 1 << (nb_samples - 1)

    return sum_result

# 示例用法
binary_array =[0, 0, 0, 0, 0, 0, 0, 1]  # 二进制补码表示
decimal_result = to_complement(binary_array)
print("Decimal result:", decimal_result)


Decimal result: -128


## 2. 二进制乘法

In [61]:
binary_input1=extract_bits_with_shift(7)
binary_input2=extract_bits_with_shift(8)
print("binary_input1:",binary_input1)
print("binary_input2:",binary_input2)

binary_input1: [1, 1, 1, 0, 0, 0, 0, 0]
binary_input2: [0, 0, 0, 1, 0, 0, 0, 0]


In [75]:
import numpy as np
# 创建一个n*2n的矩阵
n = 8
matrix = np.zeros((n, 2*n), dtype=np.uint8)
print(matrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [76]:
#计算乘法
import time
start=time.time()
for i in range(n):
    for j in range(n):
        matrix[i,i+j] = binary_input1[j] & binary_input2[i]
end=time.time()
print("计算时间：",end-start)
print(matrix)

计算时间： 9.012222290039062e-05
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [65]:
# add
def add(a, b, size):
    sum = [0] * size
    sum[0]=np.bitwise_xor(a[0],b[0])
    carry = np.bitwise_and(a[0],b[0])
    for i in range(1,size):
       tmp_s=np.bitwise_xor(a[i],b[i])
       tmp_c=np.bitwise_and(a[i],b[i])
       sum[i]=np.bitwise_xor(tmp_s,carry)
       carry=np.bitwise_or(np.bitwise_and(tmp_s,carry),tmp_c)
    return sum


In [67]:
result=[0]*n*2
for i in range(n):
    result=add(result,matrix[i],n)
result=to_complement(result)

print(result)

56


In [86]:
import time
matrix = np.zeros((n, 2*n), dtype=np.uint8)
start=time.time()
for i in range(n):
    matrix[i, i:i+n] = np.bitwise_and(binary_input2[i], binary_input1)
end=time.time()
print("计算时间：",end-start)

计算时间： 0.0008995532989501953


In [80]:
print(matrix)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [73]:
result=[0]*n*2
for i in range(n):
    result=add(result,matrix[i],n)
result=to_complement(result)

print(result)

56


## 3. 编译成concrete电路

In [84]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted","binary_input2": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ],binary_input2: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
        for j in range(n):
            matrix[i,i+j] = np.bitwise_and(binary_input1[j],binary_input2[i])
    for i in range(n):
        result=add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1,binary_input2)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)


密钥生成时间： 0.691152811050415
加密时间： 0.6923079490661621
计算时间： 9.159704446792603
解密时间： 0.0001609325408935547
[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]


In [83]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted","binary_input2": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ],binary_input2: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
         matrix[i, i:i+n] = np.bitwise_and(binary_input2[i], binary_input1)
    for i in range(n):
        result=add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1,binary_input2)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)

密钥生成时间： 0.683478593826294
加密时间： 0.6846463680267334
计算时间： 7.083902359008789
解密时间： 0.00010776519775390625
[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]


In [85]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
         matrix[i, i:i+n] = np.bitwise_and(binary_input2[i], binary_input1)
    for i in range(n):
        result=add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)

密钥生成时间： 0.690850019454956
加密时间： 0.6927173137664795
计算时间： 4.474953889846802
解密时间： 0.00011420249938964844
[0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0]


In [ ]:
from concrete import fhe
@fhe.circuit({"binary_input1": "encrypted"})
def circuit2(binary_input1: fhe.tensor[fhe.uint2, 8, ]):
    n = 8
    matrix =fhe.zeros((n, 2*n))
    result=[0]*2*n
    for i in range(n):
        for j in range(n):
            matrix[i,i+j] = np.bitwise_and(binary_input1[j],binary_input2[i])
    for i in range(n):
        result=add(result,matrix[i],2*n)
    return fhe.array(result)
import time
start=time.time()
circuit2.keygen()
end=time.time()
print("密钥生成时间：",end-start)
enc=circuit2.encrypt(binary_input1)
end=time.time()
print("加密时间：",end-start)
start=time.time()
res_enc=circuit2.run(enc)
end=time.time()
print("计算时间：",end-start)
start=time.time()
result=circuit2.decrypt(res_enc)
end=time.time()
print("解密时间：",end-start)
result=result[::-1]
print(result)

## 4. 二进制移位
将权重量化为2的幂

In [9]:
def shift_dot_product(inputs, We, cols):
    result = 0

    for i in range(cols):
        if We[i] < 0:
            for j in range(-We[i]):
                inputs[i] >>= 1
        elif We[i] > 0:
            for j in range(We[i]):
                inputs[i] <<= 1

        result += inputs[i]

    return result

# 示例用法
inputs_array = [1, 2, 3, 4, 5]
weights_array = [0, 0, 0, 0, -1]
cols_value = len(inputs_array)

result_value = shift_dot_product(inputs_array, weights_array, cols_value)
print("Result:", result_value)


Result: 12
